In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import webbrowser
import time
import json
import shutil
import os

In [ ]:
#Move downloaded APKs from Download folder to destination folder
def move_files(source_path, destin_path, dwnld_directory):
    download_wait(dwnld_directory)
    files = os.listdir(source_path)
    for file in files:
        if file.endswith('.apk'):
            try:
                shutil.move(source_path + file, destin_path)
            except:
                pass

In [ ]:
#Wait if downloading is in progress, before downloading another APK
def download_wait(dwnld_directory):
    dl_wait = True
    while dl_wait:
        time.sleep(1)
        dl_wait = False
        
        files = os.listdir(dwnld_directory)
        for fname in files:
            if fname.endswith('.crdownload'):
                dl_wait = True

In [ ]:
#API to download an APK by providing an APK's SHA256 at the end
api_key = 'https://androzoo.uni.lu/api/download?apikey=cef0598b5bd8f3e89954400a7c3ee1fcbd6a5dbf68bf5a0729ca85416d61baed&sha256='

In [ ]:
#Reading APK's SHA256, vt_detection, date and other information from csv file, provided by AndroZoo
path_to_csv = '/home/Asim/From_Windows/Adverserial ML/Datasets/AndroZoodoc/latest.csv/latest.csv'
APKs_info = pd.read_csv(path_to_csv)
APKs_info = APKs_info[APKs_info['vt_detection'].notna()]

print('csv file of shape :',APKs_info.shape)

csv file of shape : (9669639, 11)


In [ ]:
#reading family name of malware, need for family classification
labels_path = '/home/Asim/From_Windows/Adverserial ML/Datasets/AndroZoodoc/labels/names/proposed_names.json'
with open(labels_path, 'r') as proposed_names:
          labels = json.load(proposed_names)

print(f'{len(labels)} APKs having labels.')

1302969 APKs having labels.


In [ ]:
#select only APKs having vt_detection greater than 15 and less than 35
malware_geq15l35_mask = APKs_info.loc[(APKs_info['vt_detection'] >=15) & (APKs_info['vt_detection']<35)]
malware_geq15l35_shas = [sha.lower() for sha in malware_geq15l35_mask.sha256]
print(f'{len(malware_geq15l35_shas)} APKs are avialble in AndroZoo dataset having vt_detection geq15l35')
#benign_apks_mask = APKs_info['vt_detection'] == 0.0

401401 APKs are avialble in AndroZoo dataset having vt_detection geq15l35


In [ ]:
#to seperate out downloaded APKs and to download APKs. Need it in case if some APKs were downloded and some are remaining and to loop over to download the remaing APKs
dwnloded_malware_APKs = os.listdir('/home/Asim/Adverserial_ML/AndroZoo/Malware/Malware_APKs_geq35/')
dwnloded_malware_shas = [sha[:-4].lower() for sha in dwnloded_malware_APKs]
print(f'{len(dwnloded_malware_shas)} APKs are downloded having vt_detection geq15l35.')

shas_have_dwnlded = list(set(malware_geq15l35_shas).intersection(labels.keys()))
print(f'{len(have_dwnlded)} APKs are downloded and having labels.')

to_dwnld = list(set(malware_geq15l35_shas) - set(dwnloded_malware_shas))
to_dwnld = list(set(to_dwnld).intersection(labels.keys()))
print(f'{len(to_dwnld)} APKs have to download and having lables.')

22057 APKs are downloded having vt_detection geq15l35.
292291 APKs are downloded and having labels.
292291 APKs have to download and having lables.


In [ ]:
malware_shas_to_dwnld = list(set(labels.keys()).intersection(malware_geq15l35_shas))

print(f'There are {len(malware_shas_to_dwnld)} APKs to download having vt_detection greater than 5 and less than 15.')

#malware_apks_sha256 = list(malware_apks_mask['sha256'])
#print(len(malware_apks_sha256))
#benign_apks_sha256 = list(APKs_info['sha256'][benign_apks_mask])
#benign_apks_sha256 = benign_apks_sha256[16000:]
#print(len(benign_apks_sha256))

There are 1864 APKs to download having vt_detection greater than 5 and less than 15.


In [ ]:

shas_to_download = []
malware_APKs_path = '/home/Asim/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/Inter_Linear/11k_100_AEs_Testing/11k_100_APKs_AEs_Testing/'

for line in open('/home/Asim/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/Inter_Linear/11k_100_AEs_Testing/test_apks_names', 'r'):
    shas_to_download.append(line.rstrip())

tmp = [sha[:-4] for sha in os.listdir(malware_APKs_path)]
shas_to_download = list(set(shas_to_download) - set(tmp))
print(len(shas_to_download))

1


In [ ]:
shas_to_download = ['BF5EE8CF8595EE188CB1D9A93FD6C96005D3F79880AE324A82859035D58E869B', 'C681D8D39817581A62CF50899C19C96C26A6ADD59C61A41DB6ED7939524F34FF', '8FD88F160DF8BBC5602223EF46A4BA49B396A1B4929E2674190C2F6671F34831', 'A0936ED59B8D667176A7963839A621A38AD57BACF43CAC55AF0922A67A4CE8E9']


In [ ]:
#last_downloaded_sha256 = malware_apks_sha256[0]
last_downloaded_sha256 = shas_to_download[0]
count = 0

Install selenium to download APK via Google Chrome without focus. Instruction are given below

In [ ]:
#Downloading APKs
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {"safebrowsing.enabled": True})
driver = webdriver.Chrome(executable_path="/home/Asim/chromedriver_linux64/chromedriver", options=options)

for malware_sha in shas_to_download:
    driver.get(api_key + malware_sha)
    #if(count%10 == 0):
    #    move_files('/home/mujeeb/Downloads/', '/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/Inter_Linear/11k_100_AEs_Testing/', '/home/mujeeb/Downloads/')
    #last_downloaded_sha256 = malware_sha
    #count+=1

'''
for benign_sha in benign_apks_sha256:
    webbrowser.open(api_key + benign_sha)
    if(count%10 == 0):
        move_files('/home/Asim/Downloads/', '/home/Asim/Adverserial_ML/AndroZoo/Benign/Benign_APKs/', '/home/Asim/Downloads/')
    last_downloaded_sha256 = benign_sha
    count+=1'''

"\nfor benign_sha in benign_apks_sha256:\n    webbrowser.open(api_key + benign_sha)\n    if(count%10 == 0):\n        move_files('/home/mujeeb/Downloads/', '/home/mujeeb/Adverserial_ML/AndroZoo/Benign/Benign_APKs/', '/home/mujeeb/Downloads/')\n    last_downloaded_sha256 = benign_sha\n    count+=1"

## Steps for Selenium installation and to download APKs without focus

1. !pip3 install -U selenium
2. to check chrome version move to 'chrome://settings/help'
3. download corresponding version of driver from : https://sites.google.com/a/chromium.org/chromedriver/downloads
4. unzip it.
5. set path in executable_path argument.

Hope it works

6. #------add these lines before starting loop for downloads------

from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {"safebrowsing.enabled": True})
driver = webdriver.Chrome(executable_path="/home/mujeeb/chromedriver_linux64/chromedriver", options=options)

7. replace  'webbrowser.open(api_key + benign_sha)' with 'driver.get(api_key + benign_sha)' below